In [39]:
import tensorflow as tf
import numpy as np
import random
import RNNModel
import os
import multi_training
from midi_to_statematrix import midiToNoteStateMatrix
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## The goal of this project was to generate classical music using a stacked LSTM based RNN network. The snippets below show the process of importing the data in the form of midi files and the parsed by the conversion algorithm to change it into  a state matrix form.The following two snippets shows the overall structure and the method in which the underlying util functions work. The model as created and trained separately, therefore, the code below is simply to show how the data was imported and changed to a statematrix before being applied through the different layers. 

In [2]:
num_validation_pieces=10

training_pieces = multi_training.loadPieces("./music",128)

# Set aside a random set of pieces for validation purposes
validation_pieces={}
for v in range(num_validation_pieces):
    index = random.choice(list(training_pieces.keys()))
    validation_pieces[index] = training_pieces.pop(index)
    
    
print('')
print('Number of training pieces = ', len(training_pieces))    
print('Number of validation pieces = ', len(validation_pieces))     
    
    

# Generate sample Note State Matrix for dimension measurement and numerical checking purposes
# (Using external code to generate the Note State Matrix but using our own NoteInputForm (as defined in author's code) function
practice_batch_size = 5
practice_num_timesteps = 128

_, sample_state = multi_training.getPieceBatch(training_pieces, practice_batch_size, practice_num_timesteps)
sample_state = np.array(sample_state)
sample_state = np.swapaxes(sample_state, axis1=1, axis2=2)
print('Sample of State Input Batch: shape = ', sample_state.shape)

Loaded appass_2
Loaded appass_3
Loaded bach_846
Loaded bach_847
Loaded bach_850
Loaded beethoven_hammerklavier_1
Loaded beethoven_hammerklavier_1_format0
Loaded beethoven_les_adieux_1
Loaded beethoven_les_adieux_2
Loaded beethoven_opus10_2
Loaded beethoven_opus10_3
Loaded beethoven_opus22_1
Loaded beethoven_opus22_4
Loaded beethoven_opus90_2
Loaded chpn-p1
Loaded chpn-p14
Loaded chpn-p15
Loaded chpn-p16
Loaded chpn-p2
Loaded chpn-p20
Loaded chpn-p23
Loaded chpn-p3
Loaded chpn-p4
Loaded chpn-p8
Loaded chpn-p9
Loaded chpn_op10_e01
Loaded chpn_op10_e05
Loaded chpn_op10_e12
Loaded chpn_op25_e1
Loaded chpn_op25_e11
Loaded chpn_op25_e12
Loaded chpn_op25_e2
Loaded chpn_op25_e4
Loaded chpn_op27_1
Loaded chpn_op35_1
Loaded chpn_op35_3
Loaded chpn_op35_4
Loaded chpn_op66
Loaded debussy_cc_1
Loaded debussy_cc_2
Loaded debussy_cc_4
Loaded debussy_cc_6
Loaded DEB_PASS
Loaded deb_prel
Loaded mond_1
Loaded mond_3
Loaded mz_311_1
Loaded mz_311_2
Loaded mz_330_1
Loaded mz_330_3
Loaded mz_331_3
Loaded m

In [3]:
num_notes = sample_state.shape[1]
batch_size = practice_batch_size
num_timesteps = sample_state.shape[2]
midilowpitch = 18
midihighpitch = 107
Note_State_Expand = RNNModel.Input_Kernel(sample_state.astype('float32'), midilowpitch, midihighpitch, 0)
print('Note_State_Expand shape = ', Note_State_Expand.get_shape())

num_t_units=[200,200]
timewise_state_val=[]
for i in range(len(num_t_units)):
    c_t = tf.convert_to_tensor(np.zeros((batch_size*num_notes, num_t_units[i])).astype('float32'))
    h_t = tf.convert_to_tensor(np.zeros((batch_size*num_notes, num_t_units[i])).astype('float32'))
    timewise_state_val.append([h_t, c_t])

timewise_state = tuple(timewise_state_val)
timewise_out, timewise_state_out = RNNModel.LSTM_TimeWise_Training_Layer(input_data=Note_State_Expand, state_init=timewise_state, output_keep_prob=0.5)

print('Time-wise output shape = ', timewise_out.get_shape())


num_n_units = [100, 100]
notewise_state=[]
for i in range(len(num_n_units)):
    c_n = tf.convert_to_tensor(np.zeros((batch_size*num_timesteps, num_n_units[i])).astype('float32')) 
    h_n = tf.convert_to_tensor(np.zeros((batch_size*num_timesteps, num_n_units[i])).astype('float32'))
    notewise_state.append([h_n, c_n])

notewise_state=tuple(notewise_state)
y_out, note_gen_out = RNNModel.LSTM_NoteWise_Layer(timewise_out, state_init=notewise_state, output_keep_prob=0.5)

p_out = tf.sigmoid(y_out)
print('y_out shape = ', y_out.get_shape())
print('generated samples shape = ', note_gen_out.get_shape())

#print(sample_state)
#loss, log_likelihood = RNNModel.Loss_Function(sample_state.astype('float32'), y_out)
#optimizer = tf.keras.optimizers.Adadelta().minimize(loss)


Note_State_Expand shape =  (5, 90, 128, 80)
Time-wise output shape =  (5, 90, 128, 200)
y_out shape =  (5, 90, 128, 2)
generated samples shape =  (5, 90, 128, 2)
